In [1]:
import pandas as pd
import requests
import numpy
import sqlite3
import os
import time

In [2]:
# Get my building blocks
Item_URLs = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Item URLs.csv")
Resource_URLs = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Resource URLs.csv")
Artefact_Journal_URLs = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Artefact and Journal URLs.csv")
Recipies = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Albion_Equipment_Recipies_3.csv")

In [3]:
# this function lets us create a dataframe out of a list of URL's for Albion Data Project GET API.
def Get_Data(URL_list):
    df_data = pd.DataFrame(columns=['item_id', 'city', 'quality', 'sell_price_min', 'sell_price_min_date', 'sell_price_max', 'sell_price_max_date', 'buy_price_min', 'buy_price_min_date', 'buy_price_max', 'buy_price_max_date'])
    for i in range(len(URL_list)):
        Current_URL = URL_list['URL'][i]
        URL_Contents = requests.get(url=Current_URL).json()
        print(f'working on URL #{i}')
        for j in range(len(URL_Contents)):
            item = URL_Contents[j]
            item = pd.DataFrame(item, index=[0])
            df_data = pd.concat([df_data,item])
    print('API calls done')
    return df_data

# a function to take old and new prices, and drop duplicates, with the duplicate kept being the data from updated prices.
def Update_Prices(hitorical_prices, updated_prices):
    merged = pd.concat([hitorical_prices, updated_prices], ignore_index=True)
    merged.drop_duplicates(subset=['item_id', 'city', 'quality'], keep='last', inplace=True)
    return merged

In [4]:
# Make a fresh call to API to get recent data.
resource_prices_update = Get_Data(Resource_URLs)
artefact_and_journal_prices_update = Get_Data(Artefact_Journal_URLs)
time.sleep(61) #there is a 180 calls per minute limit that we must avoid. also 300 per 5 minutes limit.
item_prices_update = Get_Data(Item_URLs)


working on URL #0
working on URL #1
working on URL #2
API calls done
working on URL #0
working on URL #1
working on URL #2
working on URL #3
working on URL #4
working on URL #5
working on URL #6
working on URL #7
working on URL #8
working on URL #9
working on URL #10
working on URL #11
working on URL #12
working on URL #13
working on URL #14
working on URL #15
working on URL #16
working on URL #17
working on URL #18
working on URL #19
working on URL #20
working on URL #21
working on URL #22
working on URL #23
working on URL #24
working on URL #25
working on URL #26
working on URL #27
working on URL #28
working on URL #29
working on URL #30
working on URL #31
API calls done
working on URL #0
working on URL #1
working on URL #2
working on URL #3
working on URL #4
working on URL #5
working on URL #6
working on URL #7
working on URL #8
working on URL #9
working on URL #10
working on URL #11
working on URL #12
working on URL #13
working on URL #14
working on URL #15
working on URL #16
worki

In [5]:
#items that have no data are present but just with 0's in price data.
item_prices_update = item_prices_update[item_prices_update['sell_price_min']>0]
resource_prices_update = resource_prices_update[resource_prices_update['sell_price_min']>0]
artefact_and_journal_prices_update = artefact_and_journal_prices_update[artefact_and_journal_prices_update['sell_price_min']>0]


In [6]:
# Get prices used in last pull:
resource_prices_historical = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_resource_prices.csv")
artefact_and_journal_prices_historical = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_artefact_and_journal_prices.csv")
item_prices_historical = pd.read_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_item_prices.csv")


In [7]:
#lets update our item prices now
item_prices_final = Update_Prices(item_prices_historical, item_prices_update)
resource_prices_final = Update_Prices(resource_prices_historical, resource_prices_update)
artefact_and_journal_prices_final = Update_Prices(artefact_and_journal_prices_historical, artefact_and_journal_prices_update)


In [8]:
#and now we must save this new data into our csv files so they can be used the next time I run this program to update prices.

#top 3 are normal process, bottom 3 are to refresh the historical data files.

resource_prices_final.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_resource_prices.csv", index=False)
artefact_and_journal_prices_final.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_artefact_and_journal_prices.csv", index=False)
item_prices_final.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_item_prices.csv", index=False)

#resource_prices_update.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_resource_prices.csv", index=False)
#artefact_and_journal_prices_update.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_artefact_and_journal_prices.csv", index=False)
#item_prices_update.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\albion_item_prices.csv", index=False)

In [9]:
# Now that we have updated our prices and saved them, we can start to manipulate the price data into our report, starting with trimming columns we dont need.

item_prices_report = item_prices_final
artefact_and_journal_prices_report = artefact_and_journal_prices_final
resource_prices_report = resource_prices_final

item_prices_report = item_prices_report[['item_id','city','sell_price_min','sell_price_min_date']]
item_prices_report = item_prices_report.groupby(['item_id', 'city']).agg({'sell_price_min_date': 'min', 'sell_price_min': 'mean'}).reset_index() #items have qualities. I looked for all qualities to "widen the net" we cast with our API call. Time to group by item so that each only has 1 row.
item_prices_report['sell_price_min'] = item_prices_report['sell_price_min'].astype(int)

artefact_and_journal_prices_report = artefact_and_journal_prices_report.groupby(['item_id']).agg({'sell_price_min_date': 'min', 'sell_price_min': 'mean'}).reset_index() #these items NEVER get checked by players, and unlike items or resources, have no attachment to geography. for this reason I check all cities and group by just item to once again "widen the net"
artefact_and_journal_prices_report['sell_price_min'] = artefact_and_journal_prices_report['sell_price_min'].astype(int)

resource_prices_report = resource_prices_report[['item_id','city','sell_price_min','sell_price_min_date']]


In [10]:
# Moving to SQL for all the merges we are about to do.
Albion_DB = "../Albion.sqlite"
Albion_conn = sqlite3.connect(Albion_DB)

item_prices_report.to_sql(name='item_prices_report', if_exists='replace', con=Albion_conn)
artefact_and_journal_prices_report.to_sql(name='artefact_and_journal_prices_report', if_exists='replace', con=Albion_conn)
resource_prices_report.to_sql(name='resource_prices_report', if_exists='replace', con=Albion_conn)
Recipies.to_sql(name='Recipies', if_exists='replace', con=Albion_conn)

4240

In [11]:
report = pd.read_sql_query(
"""
with add_sell_price as (
SELECT 
Recipies.*

,item_prices_report.city as "Sell_City"
,item_prices_report.sell_price_min as "Sell_Price"
,item_prices_report.sell_price_min_date as "Sell_Price_Date"

FROM Recipies LEFT JOIN item_prices_report
ON Recipies.Item_Placeholder = item_prices_report.item_id
)
--------------------------------------------------------------------------------------------
,add_prim_res as (
SELECT 
add_sell_price.*

,resource_prices_report.city as "Main_Resource_City"
,resource_prices_report.sell_price_min as 'Main_Resource_Price'
,resource_prices_report.sell_price_min_date as 'Main_Resource_Date'
 
FROM add_sell_price LEFT JOIN resource_prices_report
ON add_sell_price.Main_Resource = resource_prices_report.item_id
)
--------------------------------------------------------------------------------------------
,add_sec_res as (
SELECT 
add_prim_res.*

,resource_prices_report.city as "Secondary_Resource_City"
,resource_prices_report.sell_price_min as 'Secondary_Resource_Price'
,resource_prices_report.sell_price_min_date as 'Secondary_Resource_Date'

FROM add_prim_res LEFT JOIN resource_prices_report
ON add_prim_res.Secondary_Resource = resource_prices_report.item_id
)
--------------------------------------------------------------------------------------------
,add_artefact as (
SELECT 
add_sec_res.*

,artefact_and_journal_prices_report.sell_price_min as 'Artefact_Price'
,artefact_and_journal_prices_report.sell_price_min_date as 'Artefact_Date'

FROM add_sec_res LEFT JOIN artefact_and_journal_prices_report
ON add_sec_res.Artefact_Placeholder = artefact_and_journal_prices_report.item_id
)
--------------------------------------------------------------------------------------------
,add_journal_empty as (
SELECT 
add_artefact.*

,artefact_and_journal_prices_report.sell_price_min as 'Journal_Empty_Price'

FROM add_artefact LEFT JOIN artefact_and_journal_prices_report
ON add_artefact.Journal_Empty = artefact_and_journal_prices_report.item_id
)
--------------------------------------------------------------------------------------------
,add_journal_full as (
SELECT 
add_journal_empty.*

,artefact_and_journal_prices_report.sell_price_min as 'Journal_Full_Price'

,CASE WHEN Artefact_Placeholder<>0 AND Artefact_Price IS NULL THEN 1 ELSE 0 END AS 'Missing_Artefact'
,CASE WHEN Secondary_Resource_Quantity<>'0' AND Secondary_Resource_Price IS NULL THEN 1 ELSE 0 END AS 'Missing_Second_Resource'

FROM add_journal_empty LEFT JOIN artefact_and_journal_prices_report
ON add_journal_empty.Journal_Full = artefact_and_journal_prices_report.item_id
)
--------------------------------------------------------------------------------------------

SELECT
*

,CASE WHEN Secondary_Resource_Price IS NULL THEN 0 ELSE Secondary_Resource_Price END as 'Secondary_Resource_Price_nn'
,CASE WHEN Artefact_Price IS NULL THEN 0 ELSE Artefact_Price END as 'Artefact_Price_nn'
,Tier||'.'||Enchant as Tier_Enchant

FROM add_journal_full
WHERE Sell_Price IS NOT NULL AND Missing_Artefact=0 AND Missing_Second_Resource=0 AND Main_Resource_Price>0 AND Journal_Empty_Price IS NOT NULL AND Journal_Full_Price IS NOT NULL AND Sell_City="Black Market"




 """, Albion_conn)

In [12]:
report.columns

Index(['index', 'Item_Placeholder', 'Tier', 'Enchant', 'Item_Name',
       'Main_Resource', 'Main Resource Quantity', 'Secondary_Resource',
       'Secondary_Resource_Quantity', 'Artefact_Placeholder',
       'Artefact_Quantity', 'City_Bonus', 'Crafting Fame', 'Journal_Empty',
       'Journal_Full', 'Journal_EXP_Fill', 'Focus Cost', 'Sell_City',
       'Sell_Price', 'Sell_Price_Date', 'Main_Resource_City',
       'Main_Resource_Price', 'Main_Resource_Date', 'Secondary_Resource_City',
       'Secondary_Resource_Price', 'Secondary_Resource_Date', 'Artefact_Price',
       'Artefact_Date', 'Journal_Empty_Price', 'Journal_Full_Price',
       'Missing_Artefact', 'Missing_Second_Resource',
       'Secondary_Resource_Price_nn', 'Artefact_Price_nn', 'Tier_Enchant'],
      dtype='object')

In [13]:
# moving back to Pandas for final non-merge operations

report['Secondary_Resource_Price'] = report['Secondary_Resource_Price_nn']
report = report.drop('Secondary_Resource_Price_nn', axis=1)

report['Artefact_Price'] = report['Artefact_Price_nn']
report = report.drop('Artefact_Price_nn', axis=1)

report['Resource_Costs'] = (report['Main Resource Quantity']*report['Main_Resource_Price'])+(report['Secondary_Resource_Quantity']*report['Secondary_Resource_Price'])
report['Artefact_Costs'] = report['Artefact_Quantity']*report['Artefact_Price']
report['Journal_Profit'] = ((report['Journal_Full_Price']*0.975)-report['Journal_Empty_Price'])*(report['Crafting Fame']/report['Journal_EXP_Fill'])

report['Profit_City'] = (report['Sell_Price']*0.945)-report['Artefact_Costs']-(report['Resource_Costs']*0.752)+report['Journal_Profit']
report['Profit_City_Focus'] = (report['Sell_Price']*0.945)-report['Artefact_Costs']-(report['Resource_Costs']*0.502)+report['Journal_Profit']
report['Profit_City_BZ_Q6'] = (report['Sell_Price']*0.945)-report['Artefact_Costs']-(report['Resource_Costs']*0.598)+report['Journal_Profit']


report = report[['Item_Name','City_Bonus','Tier','Enchant','Tier_Enchant','Profit_City','Profit_City_Focus','Profit_City_BZ_Q6','Sell_City','Sell_Price','Sell_Price_Date','Main_Resource','Main_Resource_Price','Main_Resource_City','Main_Resource_Date','Secondary_Resource','Secondary_Resource_Price','Secondary_Resource_City','Secondary_Resource_Date','Artefact_Costs','Journal_Profit','Resource_Costs']]

In [14]:
report.columns

Index(['Item_Name', 'City_Bonus', 'Tier', 'Enchant', 'Tier_Enchant',
       'Profit_City', 'Profit_City_Focus', 'Profit_City_BZ_Q6', 'Sell_City',
       'Sell_Price', 'Sell_Price_Date', 'Main_Resource', 'Main_Resource_Price',
       'Main_Resource_City', 'Main_Resource_Date', 'Secondary_Resource',
       'Secondary_Resource_Price', 'Secondary_Resource_City',
       'Secondary_Resource_Date', 'Artefact_Costs', 'Journal_Profit',
       'Resource_Costs'],
      dtype='object')

In [15]:
report

,Item_Name,City_Bonus,Tier,Enchant,Tier_Enchant,Profit_City,Profit_City_Focus,Profit_City_BZ_Q6,Sell_City,Sell_Price,...,Main_Resource_Price,Main_Resource_City,Main_Resource_Date,Secondary_Resource,Secondary_Resource_Price,Secondary_Resource_City,Secondary_Resource_Date,Artefact_Costs,Journal_Profit,Resource_Costs
0,Adept's Great Arcane Staff,Lymhurst,4,0,4.0,3250.573000,4304.573000,3899.837000,Black Market,6647,...,134,Bridgewatch,2023-04-28T15:35:00,T4_METALBAR,128,Bridgewatch,2023-04-28T15:50:00,0,139.590000,4216
1,Adept's Great Arcane Staff,Lymhurst,4,0,4.0,3232.525000,4292.525000,3885.485000,Black Market,6647,...,134,Bridgewatch,2023-04-28T15:35:00,T4_METALBAR,130,Fort Sterling,2023-04-28T14:15:00,0,139.590000,4240
2,Adept's Great Arcane Staff,Lymhurst,4,0,4.0,3196.429000,4268.429000,3856.781000,Black Market,6647,...,134,Bridgewatch,2023-04-28T15:35:00,T4_METALBAR,134,Lymhurst,2023-04-28T12:00:00,0,139.590000,4288
3,Adept's Great Arcane Staff,Lymhurst,4,0,4.0,3313.741000,4346.741000,3950.069000,Black Market,6647,...,134,Bridgewatch,2023-04-28T15:35:00,T4_METALBAR,121,Martlock,2023-04-28T01:25:00,0,139.590000,4132
4,Adept's Great Arcane Staff,Lymhurst,4,0,4.0,3476.173000,4455.173000,4079.237000,Black Market,6647,...,134,Bridgewatch,2023-04-28T15:35:00,T4_METALBAR,103,Thetford,2023-04-28T15:40:00,0,139.590000,3916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26452,Elder's Graveguard Boots,Martlock,8,0,8.0,92175.615662,125459.615662,112678.559662,Black Market,238985,...,16642,Bridgewatch,2023-04-28T13:15:00,None,0,None,None,64930,31383.062662,133136
26453,Elder's Graveguard Boots,Martlock,8,0,8.0,87049.983662,122037.983662,108602.591662,Black Market,238985,...,17494,Fort Sterling,2023-04-28T11:10:00,None,0,None,None,64930,31383.062662,139952
26454,Elder's Graveguard Boots,Martlock,8,0,8.0,81316.735662,118210.735662,104043.439662,Black Market,238985,...,18447,Lymhurst,2023-04-28T12:00:00,None,0,None,None,64930,31383.062662,147576
26455,Elder's Graveguard Boots,Martlock,8,0,8.0,90443.007662,124303.007662,111300.767662,Black Market,238985,...,16930,Martlock,2023-04-27T21:25:00,None,0,None,None,64930,31383.062662,135440


In [16]:
report.to_csv(r"C:\Users\jorda\Desktop\Albion_Crafting_Report_Python\Market_Data\report.csv", index=False)